<a href="https://colab.research.google.com/github/flaghunter21/DFS/blob/main/FanDuel_NFL_Pick3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FanDuel NFL Pick 3 Lineup Combinations

In [2]:
import pandas as pd
import itertools
import numpy as np

In [3]:
pick3_df = pd.read_csv('/content/FanDuel-NFL-2022 ET-12 ET-11 ET-84418-players-list.csv')
pick3_df.columns
pick3_df.head()

,Id,Position,First Name,Nickname,Last Name,FPPG,Played,Salary,Game,Team,Opponent,Injury Indicator,Injury Details,Tier,Unnamed: 14,Unnamed: 15,Roster Position
0,84418-57439,QB,Patrick,Patrick Mahomes,Mahomes,25.718333,12.0,5,KC@DEN,KC,DEN,NaN,NaN,NaN,NaN,NaN,MVP - 1.5X Points/AnyFLEX
1,84418-80001,RB,Austin,Austin Ekeler,Ekeler,18.816667,12.0,5,MIA@LAC,LAC,MIA,NaN,NaN,NaN,NaN,NaN,MVP - 1.5X Points/AnyFLEX
2,84418-39280,RB,Derrick,Derrick Henry,Henry,17.671666,12.0,5,JAC@TEN,TEN,JAC,NaN,NaN,NaN,NaN,NaN,MVP - 1.5X Points/AnyFLEX
3,84418-53681,WR,Tyreek,Tyreek Hill,Hill,18.241666,12.0,5,MIA@LAC,MIA,LAC,NaN,NaN,NaN,NaN,NaN,MVP - 1.5X Points/AnyFLEX
4,84418-85671,WR,Justin,Justin Jefferson,Jefferson,18.288334,12.0,5,MIN@DET,MIN,DET,NaN,NaN,NaN,NaN,NaN,MVP - 1.5X Points/AnyFLEX


In [4]:
pick3_df = pick3_df.loc[:, ['Id','Position', 'Nickname', 'FPPG', 'Salary', 'Team']]
pick3_df.head()

,Id,Position,Nickname,FPPG,Salary,Team
0,84418-57439,QB,Patrick Mahomes,25.718333,5,KC
1,84418-80001,RB,Austin Ekeler,18.816667,5,LAC
2,84418-39280,RB,Derrick Henry,17.671666,5,TEN
3,84418-53681,WR,Tyreek Hill,18.241666,5,MIA
4,84418-85671,WR,Justin Jefferson,18.288334,5,MIN


In [5]:
combos = list(itertools.combinations(pick3_df['Nickname'], 3))
print(combos[:5])
print(len(combos))

[('Patrick Mahomes', 'Austin Ekeler', 'Derrick Henry'), ('Patrick Mahomes', 'Austin Ekeler', 'Tyreek Hill'), ('Patrick Mahomes', 'Austin Ekeler', 'Justin Jefferson'), ('Patrick Mahomes', 'Austin Ekeler', 'A.J. Brown'), ('Patrick Mahomes', 'Austin Ekeler', 'Joe Mixon')]
62467440


In [6]:
combos_df = pd.DataFrame(combos, columns=['FLEX1','FLEX2','FLEX3'])
combos_df.head()

,FLEX1,FLEX2,FLEX3
0,Patrick Mahomes,Austin Ekeler,Derrick Henry
1,Patrick Mahomes,Austin Ekeler,Tyreek Hill
2,Patrick Mahomes,Austin Ekeler,Justin Jefferson
3,Patrick Mahomes,Austin Ekeler,A.J. Brown
4,Patrick Mahomes,Austin Ekeler,Joe Mixon


In [7]:
projection_map = dict(zip(pick3_df['Nickname'], pick3_df['FPPG']))
salary_map = dict(zip(pick3_df['Nickname'], pick3_df['Salary']))

In [8]:
proj_cols = []
salary_cols = []
for i in range(1,4):
  combos_df[f'FLEX{i}_Proj'] = combos_df[f'FLEX{i}'].map(projection_map)
  proj_cols.append(f'FLEX{i}_Proj')
for i in range(1, 4):
  combos_df[f'FLEX{i}_Salary'] = combos_df[f'FLEX{i}'].map(salary_map)
  salary_cols.append(f'FLEX{i}_Salary')
     

In [9]:
combos_df.head()

,FLEX1,FLEX2,FLEX3,FLEX1_Proj,FLEX2_Proj,FLEX3_Proj,FLEX1_Salary,FLEX2_Salary,FLEX3_Salary
0,Patrick Mahomes,Austin Ekeler,Derrick Henry,25.718333,18.816667,17.671666,5,5,5
1,Patrick Mahomes,Austin Ekeler,Tyreek Hill,25.718333,18.816667,18.241666,5,5,5
2,Patrick Mahomes,Austin Ekeler,Justin Jefferson,25.718333,18.816667,18.288334,5,5,5
3,Patrick Mahomes,Austin Ekeler,A.J. Brown,25.718333,18.816667,14.625000,5,5,4
4,Patrick Mahomes,Austin Ekeler,Joe Mixon,25.718333,18.816667,16.039999,5,5,4


In [10]:
combos_df['Total Proj'] = combos_df[proj_cols].sum(axis=1)
combos_df['Salary Cap'] = combos_df[salary_cols].sum(axis=1)
combos_df.head()

,FLEX1,FLEX2,FLEX3,FLEX1_Proj,FLEX2_Proj,FLEX3_Proj,FLEX1_Salary,FLEX2_Salary,FLEX3_Salary,Total Proj,Salary Cap
0,Patrick Mahomes,Austin Ekeler,Derrick Henry,25.718333,18.816667,17.671666,5,5,5,62.206666,15
1,Patrick Mahomes,Austin Ekeler,Tyreek Hill,25.718333,18.816667,18.241666,5,5,5,62.776666,15
2,Patrick Mahomes,Austin Ekeler,Justin Jefferson,25.718333,18.816667,18.288334,5,5,5,62.823334,15
3,Patrick Mahomes,Austin Ekeler,A.J. Brown,25.718333,18.816667,14.625000,5,5,4,59.160000,14
4,Patrick Mahomes,Austin Ekeler,Joe Mixon,25.718333,18.816667,16.039999,5,5,4,60.574999,14


In [11]:
print(len(combos_df))
combos_df = combos_df[combos_df['Salary Cap'] <= 7]
print(len(combos_df))
combos_df.head()

62467440
60817184


,FLEX1,FLEX2,FLEX3,FLEX1_Proj,FLEX2_Proj,FLEX3_Proj,FLEX1_Salary,FLEX2_Salary,FLEX3_Salary,Total Proj,Salary Cap
83439,Patrick Mahomes,Tyler Vaughns,Quintin Morris,25.718333,NaN,1.700000,5,1,1,27.418333,7
83440,Patrick Mahomes,Tyler Vaughns,River Cracraft,25.718333,NaN,3.671429,5,1,1,29.389762,7
83441,Patrick Mahomes,Tyler Vaughns,Jameson Williams,25.718333,NaN,0.000000,5,1,1,25.718333,7
83442,Patrick Mahomes,Tyler Vaughns,Ricky Seals-Jones,25.718333,NaN,4.918182,5,1,1,30.636515,7
83443,Patrick Mahomes,Tyler Vaughns,Shemar Bridges,25.718333,NaN,NaN,5,1,1,25.718333,7


In [12]:
combos_df = combos_df.sort_values(by='Total Proj', ascending=False)

In [13]:
combos_df = combos_df[1:1000]
combos_df.to_csv('pick3_lineups.csv')